# MARCH MADNESS PREDICTOR

Authors: Connor Finn, Riley Greene <br>
Date: 1/24/20 <br>
Warren Buffet is still paying 1 billion for a perfect bracket

In [16]:
# our imports for the model
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup 
import lxml.html as lh

# a few constants
start_year = 2009

Because basketball has changed so much over the past years, we believe that data prior to the year 2009 could be detrimental to our model. For this reason, the next block of code will be used to slim the data down from the 1985 - 2018 seasons to a dataframe including the the 2020 - 2018 NCAA tournaments.

In [17]:
# read ncaa tournament results into dataframe ( data obtained from Kaggle)
ncaa_total = pd.read_csv('NCAATourneyCompactResults.csv')  
# read team ID dataframe (from Kaggle)
team_data = pd.read_csv('teams.csv')  
# get the first index for the desired year
start = min(ncaa_total[ncaa_total.iloc[:,0] == start_year].index.values.astype(int)) 
# shorten our dataframe
ncaa_short = ncaa_total.iloc[start: , :]

The team_id provides names written as 'Penn State'. The data we will be collecting is from Sports-reference.com. The url for penn state's data is 'https://www.sports-reference.com/cbb/schools/penn-state/2020.html' <br>
1) the name needs to be adjusted to no spaces, all lower caps <br>
2) the overall framework is 'https://www.sports-reference.com/cbb/schools/NAME/YEAR.html'<br>

In [18]:
"""
* this function will be used to transform a provided name to one that can be input into a sports-reference url
* this will likely need to be adjusted as new teams are included
* this needs a unit test (i.e. run all our team names through once to see if they pass)
"""

# this is soooo bad
def clean_team_name(name):
    # name: string
    team = name
    team = '!' + team + '!'                   # first and last character
    team = team.replace(' ' , '-')            # no spaces
    team = team.replace('(' , '')
    team = team.replace(')' , '')
    team = team.replace('.' , '')
    team = team.replace( "'" , '')
    team = team.replace("&" , "")
    team = team.lower()
    team = team.replace('!southern-univ!' , '!southern!')
    team = team.replace('!w-texas-am!' , '!west-texas-am!')
    team = team.replace('!armstrong-st!' , '!armstrong!')
    team = team.replace('ark-' , 'arkansas-')
    team = team.replace('-st!' , '-state!')
    team = team.replace('!n-' , '!north-')
    team = team.replace('!e-' , '!eastern-')
    team = team.replace('!cs-' , '!cal-state-')
    team = team.replace('!s-illinois!' , '!southern-illinois!')
    team = team.replace('!s-' , '!south-')
    team = team.replace('!w-' , '!western-')
    team = team.replace('!nc-' , '!north-carolina-')
    team = team.replace('chr!' , 'christian!')
    team = team.replace('-so!' , '-southern!')
    team = team.replace('!fl-' , '!florida-')
    team = team.replace('!uc-' , '!california-')
    team = team.replace('intl!' , 'international!')
    team = team.replace('cal-state-sacramento' , 'sacramento-state')
    team = team.replace('univ!' , 'university!')
    team = team.replace('miss!' , 'mississippi!')
    team = team.replace('-slo' , '')
    team = team.replace('car!' , 'carolina!')
    team = team.replace('!ne-omaha!' , '!nebraska-omaha!')
    team = team.replace('!ne-' , '!northeastern-')
    team = team.replace('!ut-' , '!texas-')
    team = team.replace('okla-' , 'oklahoma-')
    team = team.replace('!penn!' , '!pennsylvania!')
    team = team.replace('!unc' , '!north-carolina-')
    team = team.replace('!american-university!' , '!american!')
    team = team.replace('!detroit!' , '!detroit-mercy!')
    team = team.replace('!loy-' , '!loyola-')
    team = team.replace('!loyola-chicago!' , '!loyola-il!')
    team = team.replace('!north-kentucky!' , '!northern-kentucky!')
    team = team.replace('!north-illinois!' , '!northern-illinois!')
    team = team.replace('!north-colorado!' , '!northern-colorado!')
    team = team.replace('!cal-baptist!' , '!california-baptist!')
    team = team.replace('!augusta!' , '!augusta-state!')
    team = team.replace('!etsu!' , '!east-tennessee-state!')
    team = team.replace('!mt-' , '!mount-')
    team = team.replace('!g-washington!' , '!george-washington!')
    team = team.replace('!ga-' , '!georgia-')
    team = team.replace('!il-' , '!illinois-')
    team = team.replace('!houston-bap!' , '!houston-baptist!')
    team = team.replace('!kennesaw!' , '!kennesaw-state!')
    team = team.replace('!bowling-green!' , '!bowling-green-state!')
    team = team.replace('!col-charleston!' , '!college-of-charleston!')
    team = team.replace('!cent-arkansas!' , '!central-arkansas!')
    team = team.replace('!central-conn!' , '!central-connecticut-state!')
    team = team.replace('!kent!' , '!kent-state!')
    team = team.replace('!lsu!' , '!louisiana-state!')
    team = team.replace('!ms-' , '!mississippi-')
    team = team.replace('!f-dickinson!' , '!fairleigh-dickinson!')
    team = team.replace('!byu!' , '!brigham-young!')
    team = team.replace('!ma-' , '!massachusetts-')
    team = team.replace('!northwestern-la!' , '!northwestern-state!')
    team = team.replace('!long-island!' , '!long-island-university!')
    team = team.replace('!wi-' , '!')
    team = team.replace('!c-' , '!central-')
    team = team.replace('!md-e-shore!' , '!maryland-eastern-shore!')
    team = team.replace('!st-johns!' , '!st-johns-ny!')
    team = team.replace('!tcu!' , '!texas-christian!')
    team = team.replace('!tx-' , '!texas-')
    team = team.replace('!va-' , '!virginia-')
    team = team.replace('!vmi!' , '!virginia-millitary-institute!')
    team = team.replace('!wku!' , '!western-kentucky!')
    team = team.replace('!utep!' , '!texas-el-paso!')
    team = team.replace('!st-marys-ca!' , '!saint-marys-ca!')
    team = team.replace('!santa-barbara!' , '!california-santa-barbara!')
    team = team.replace('!unlv!' , '!nevada-las-vegas!')
    team = team.replace('-pa!' , '!')
    team = team.replace('!uab!' , '!alabama-birmingham!')
    team = team.replace('!mtsu!' , '!middle-tennessee!')
    team = team.replace('!smu!' , '!southern-methodist!')
    team = team.replace('!sf-austin!' , '!stephen-f-austin!')
    team = team.replace('!st-josephs!' , '!saint-josephs!')
    team = team.replace('!umbc!' , '!maryland-baltimore-county!')
    team = team.replace('!usc!' , '!southern-california!')
    team = team.replace('!st-peters!' , '!saint-peters!')
    team = team.replace('!st-louis!' , '!saint-louis!')
    team = team.replace('!ull!' , '!louisiana-lafayette!')
    team = team.replace('!usc!' , '!southern-california!')

    
    

    team = team.replace('--' , '-')
    team = team.replace("university-of-" , "")
    
    
    team = team[1:-1]
    
    return team
def test_team_names(team_names):
    # team_names: list of strings 
    errors = 0
    for name in team_names:
        url_test =  "https://www.sports-reference.com/cbb/schools/" + name + "/index.html"
        page_test = requests.get(url_test)   # scrape
        soup_test = BeautifulSoup(page_test.content, "lxml")   # parse
        tables_test = soup_test.findAll('table')    # search for tables 
        if len(tables_test) != 0:
            print("                                 " , name , " Pass")
        else:
            print(name , " Fail")
            errors += 1
    print("finished unit test. There were " , errors , " errors.")
    return errors

team_data is a dataframe which has the columns: 'TeamID', 'TeamName', 'FirstD1Season', 'LastD1Season'.  We want to create a new column which has the team names in accordance to the sports-reference.com framework. 

First, reduce the team_data dataframe to only those teams who actually compteded in the tournament

In [19]:
s1 = set(team_data.TeamID)
s2 = set(ncaa_short.WTeamID.append(ncaa_short.LTeamID))
extra_teams = s1.symmetric_difference(s2)
for value in extra_teams:
    team_data = team_data.drop(team_data[team_data["TeamID"] == value].index)


In [20]:
# get a list of all the team names in our dataframe
team_names = team_data.TeamName.values.tolist()

# clean every name in the team_data dataframe
sr_names = []
for name in team_names:
    sr_names += [clean_team_name(name)]
# run the test: if no error messages show up, then we add the list to dataframe
#num_errors = test_team_names(sr_names)
num_errors = 0
if num_errors ==0:
    team_data["SrNames"] = sr_names

Goal: compile season long data for the teams who competed in the 2010 - 2018 NCAA tournaments. The idea is that the season long data is the information we will have in the future for creating predictions.<br>
- in the ncaa_short, we have the year the game was played, and the id's of both teams
- in team_data we have the team id, along with the team name (cleaned for sports-reference use).

In [21]:
# get arrays which include the team years and id's 
winner_array = np.vstack((ncaa_short.Season.values , ncaa_short.WTeamID.values ))
loser_array = np.vstack((ncaa_short.Season.values , ncaa_short.LTeamID.values))

# List of years
years = ncaa_short.Season.unique()

# the below loop will create a 2 x n array of all unique teams (year , school_id) which competed in NCAA's
all_teams = np.empty((2,0)) # fill this array
a_team_list = []
for year in years:
    temp_1 = ncaa_short.WTeamID.loc[(ncaa_short.Season == year)].append(ncaa_short.LTeamID.loc[(ncaa_short.Season == year)]).unique()
    a_team_list = a_team_list + temp_1.tolist()
    temp_2 = np.full(shape = len(temp_1), fill_value = year , dtype = np.int)
    temp_3 = np.vstack((temp_2 , temp_1)) 
    all_teams = np.hstack((all_teams , temp_3)) # fill array


We now want to build a dataframe which has a list of the teams and their season long statistics. <br> 
all_teams is now a 2xn array of unique teams (year, school_id). We will gain access to the team name using: <br>
- team_name = team_data.SrNames.loc[(team_data.TeamID == INSERT TEAM ID HERE)].values.tolist()[0]

In [13]:
# this will collect the season data for every team we selected in the list, for the number of indicated years
seasonStats = pd.DataFrame()   # start with an empty dataframe

for i in range(all_teams.shape[1]):   # this is the years 
	team_name = team_data.SrNames.loc[(team_data.TeamID == all_teams[1 , i])].values.tolist()[0] # get the team Name
	url3 = "https://www.sports-reference.com/cbb/schools/" + str(team_name) + "/" + str(int(all_teams[0 , i])) + ".html" # season data
	pageSe = requests.get(url3)   # scrape
	soupSe = BeautifulSoup(pageSe.content, "lxml")   # parse
	tablesSe = soupSe.findAll('table')    # search for tables 
	# convert to a dataframe and label the data
	print("Team number " ,  i)   # this shows progress
	dfSe = pd.read_html(str(tablesSe[1]))[0]      # select the table of interest into a pandas dataframe
	dfSe.drop([1 , 2, 3] , inplace=True)	      # only want the team data. (although might consider fouls against too)
	print(team_name)    
	dfSe = dfSe.replace('Team' , team_name)   # want the name of the team 
	numRows = len(dfSe.index) # get the number of rows...... should just be one here
	dfSe['Team_ID'] = int(all_teams[1 , i])        # put in the team ID's
# 	dfSe['Conf_ID'] = powerSixTeamsAndConf[j][1]        # put in the conference ID's  
# 	dfSe['Conf'] = powerSixConf[powerSixTeamsAndConf[j][1] - 1]       # put in the team's conference
	dfSe['Date'] = all_teams[0 , i]  # only want the year in this column
	seasonStats = pd.concat([seasonStats, dfSe] ,  axis=0) # add this to the season stats empty dataframe we started with
seasonStats = seasonStats.rename(columns={'Unnamed: 0': 'Team'})  # simply clean up the columns 
    
    

Team number  0
morehead-state
Team number  1
connecticut
Team number  2
duke
Team number  3
gonzaga
Team number  4
louisiana-state
Team number  5
maryland
Team number  6
memphis
Team number  7
michigan
Team number  8
north-carolina
Team number  9
oklahoma
Team number  10
purdue
Team number  11
texas
Team number  12
texas-am
Team number  13
ucla
Team number  14
villanova
Team number  15
western-kentucky
Team number  16
washington
Team number  17
arizona
Team number  18
arizona-state
Team number  19
cleveland-state
Team number  20
dayton
Team number  21
kansas
Team number  22
louisville
Team number  23
marquette
Team number  24
michigan-state
Team number  25
missouri
Team number  26
oklahoma-state
Team number  27
pittsburgh
Team number  28
siena
Team number  29
syracuse
Team number  30
southern-california
Team number  31
wisconsin
Team number  32
xavier
Team number  33
alabama-state
Team number  34
chattanooga
Team number  35
binghamton
Team number  36
akron
Team number  37
butler
Team n

Team number  292
iowa-state
Team number  293
kansas
Team number  294
miami-fl
Team number  295
minnesota
Team number  296
mississippi
Team number  297
north-carolina
Team number  298
ohio-state
Team number  299
san-diego-state
Team number  300
temple
Team number  301
liberty
Team number  302
middle-tennessee
Team number  303
long-island-university
Team number  304
boise-state
Team number  305
belmont
Team number  306
bucknell
Team number  307
nevada-las-vegas
Team number  308
missouri
Team number  309
southern
Team number  310
new-mexico
Team number  311
davidson
Team number  312
south-dakota-state
Team number  313
valparaiso
Team number  314
oklahoma-state
Team number  315
new-mexico-state
Team number  316
montana
Team number  317
akron
Team number  318
pittsburgh
Team number  319
cincinnati
Team number  320
albany-ny
Team number  321
georgetown
Team number  322
northwestern-state
Team number  323
colorado
Team number  324
notre-dame
Team number  325
western-kentucky
Team number  326


Team number  580
florida-gulf-coast
Team number  581
south-dakota-state
Team number  582
nevada
Team number  583
minnesota
Team number  584
vanderbilt
Team number  585
princeton
Team number  586
vermont
Team number  587
virginia-commonwealth
Team number  588
north-carolina-wilmington
Team number  589
bucknell
Team number  590
virginia-tech
Team number  591
maryland
Team number  592
seton-hall
Team number  593
new-mexico-state
Team number  594
troy
Team number  595
northern-kentucky
Team number  596
jacksonville-state
Team number  597
oklahoma-state
Team number  598
miami-fl
Team number  599
texas-southern
Team number  600
iona
Team number  601
creighton
Team number  602
marquette
Team number  603
kent-state
Team number  604
southern-methodist
Team number  605
dayton
Team number  606
radford
Team number  607
st-bonaventure
Team number  608
syracuse
Team number  609
texas-southern
Team number  610
alabama
Team number  611
buffalo
Team number  612
duke
Team number  613
florida
Team number

In [22]:
teamIDs = pd.unique(all_teams[1])

In [46]:
# a loop to get the win loss for the teams
WLTeams1 = pd.DataFrame()      # initialize and empty dataframe

team_name = team_data.SrNames.loc[(team_data.TeamID == teamIDs[0])].values.tolist()[0]
url3 = "https://www.sports-reference.com/cbb/schools/" + team_name + "/"
teamNames = team_name
        # get the data
pageWL = requests.get(url3)
soupWL = BeautifulSoup(pageWL.content, "lxml")
tablesWL = soupWL.findAll('table')
        
        # build a dataframe of the win loss data
win_loss_df = pd.read_html(str(tablesWL))[0]
win_loss_6_df = win_loss_df.head(12)
win_loss_6_df['team'] = teamNames

WLTeams1 = WLTeams1.append(win_loss_6_df , ignore_index=True)    
# this will get the W-L%
WL = win_loss_df.iloc[:, [5]]

WL20 = WL.values[1:6]
        # this gets rid of the wierd index This should be done in the dataframe ahead of time 
index = np.argwhere(WL20=='W-L%')
WL20 = np.delete(WL20, index)
    
# # Add a column of the years to the win loss dataframe
years = [2020 , 2019 , 2018 , 2017 , 2016 , 2015 , 2014 , 2013 , 2012 , 2011 , 2010 , 2009 ] * 1     
WLTeams1['year'] = years
# # going to need to use an intersection later to add the win loss record for each team for each year to the dataframe

/Users/connorfinn/Desktop/march_madness/march_madness/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [45]:

# a loop to get the win loss for the teams
WLTeams = pd.DataFrame()      # initialize and empty dataframe
for j in range(len(teamIDs)): 
    team_name = team_data.SrNames.loc[(team_data.TeamID == teamIDs[j])].values.tolist()[0]
    url3 = "https://www.sports-reference.com/cbb/schools/" + team_name + "/"
        # get the data
    pageWL = requests.get(url3)
    soupWL = BeautifulSoup(pageWL.content, "lxml")
    tablesWL = soupWL.findAll('table')
        
        # build a dataframe of the win loss data
    win_loss_df = pd.read_html(str(tablesWL))[0]
    win_loss_6_df = win_loss_df.head(12)
    win_loss_6_df['team'] = team_name

    WLTeams = WLTeams.append(win_loss_6_df , ignore_index=True)    
# this will get the W-L%
    WL = win_loss_df.iloc[:, [5]]

    WL20 = WL.values[1:6]
    # this gets rid of the wierd index This should be done in the dataframe ahead of time 
    index = np.argwhere(WL20=='W-L%')
    WL20 = np.delete(WL20, index)
    


# going to need to use an intersection later to add the win loss record for each team for each year to the dataframe

/Users/connorfinn/Desktop/march_madness/march_madness/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/connorfinn/Desktop/march_madness/march_madness/lib/python3.7/site-packages/ipykernel_launcher.py:22: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


Fix the dataframe columns

In [84]:
#WLTeams.columns = WLTeams.columns.droplevel(level=0)
WLTeams['team'] = WLTeams[""]
WLTeams.drop(""  , axis=1)

,Rk,Season,Conf,W,L,W-L%,W,L,W-L%,SRS,SOS,PTS,PTS,AP Pre,AP High,AP Final,NCAA Tournament,Seed,Coach(es),team
0,1,2019-20,OVC,11,12,.478,5,5,.500,-10.64,-7.12,70.6,69.3,NaN,NaN,NaN,NaN,NaN,Preston Spradlin (11-12),morehead-state
1,2,2018-19,OVC,13,20,.394,8,10,.444,-7.73,-1.76,73.7,77.1,NaN,NaN,NaN,NaN,NaN,Preston Spradlin (13-20),morehead-state
2,3,2017-18,OVC,8,21,.276,4,14,.222,-10.65,-1.91,70.4,76.0,NaN,NaN,NaN,NaN,NaN,Preston Spradlin (8-21),morehead-state
3,4,2016-17,OVC,14,16,.467,10,6,.625,-3.60,-1.85,79.0,77.0,NaN,NaN,NaN,NaN,NaN,"Preston Spradlin (12-9), Sean Woods (2-7)",morehead-state
4,5,2015-16,OVC,23,14,.622,11,5,.688,1.04,-1.65,73.6,69.0,NaN,NaN,NaN,NaN,NaN,Sean Woods (23-14),morehead-state
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2595,8,2012-13,Big 12,11,21,.344,2,16,.111,-4.27,4.41,55.0,62.6,NaN,NaN,NaN,NaN,NaN,Trent Johnson (11-21),texas-christian
2596,9,2011-12,MWC,18,15,.545,7,7,.500,1.72,3.44,72.2,72.1,NaN,NaN,NaN,NaN,NaN,Jim Christian (18-15),texas-christian
2597,10,2010-11,MWC,11,22,.333,1,15,.063,0.96,3.93,67.3,70.2,NaN,NaN,NaN,NaN,NaN,Jim Christian (11-22),texas-christian
2598,11,2009-10,MWC,13,19,.406,5,11,.313,-0.25,4.15,67.8,70.9,NaN,NaN,NaN,NaN,NaN,Jim Christian (13-19),texas-christian


Save the dataframes locally to CSV.

In [85]:
seasonStats.to_csv('season_data.csv' , index=False)
WLTeams.to_csv('more_team_data.csv' , index=False)
team_data.to_csv('team_list.csv' , index=False)
ncaa_short.to_csv('ncaa_short.csv' , index = False)